## Семинар 7. Извлечение именованных сущностей.

Именованные сущности - общий термин, который используют для обозначения какого-то множества слов (словосочетаний, последовательностей символов), которые представляют какой-то особый инетерес в контексте решаемой практической задачи и которые нужно отделить от остальных слов. Это могут быть классические имена, фамилии, названия организаций, городов, рек, гор, а могут быть и, например, денежные суммы, даты, статьи кодексов, номера телефонов.


В многих случаях случаях можно написать регуляру и решиьть задачу на приемлимом уровне (например собрать все номера телефонов совсем несложно). Но когда сущности выраются нестандартным образом, постоянно расширяются и меняются по формам, задача становится очень сложной.

К тому же в разных текстах употребляются разные сущности. Поэтому универсальных извлекателей сущностей нет. Есть только стандартные Персоны, Локации, Организации. 

Для английского удобно использовать spacy. Там сразу извлекаются сущности с хорошим качеством.

Для русского (если не хочется ничего делать) можно использовать тэги из pymorphy.

In [15]:
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [25]:
p = morph.parse('Михаил')[0].tag
print('Тэги - ', p)
print('Name' in p) #тэг имени

Тэги -  NOUN,anim,masc,Name sing,nomn
True


In [27]:
p = morph.parse('Иванов')[0].tag
print('Тэги - ', p)
print('Surn' in p) #тэг фамилии

Тэги -  NOUN,anim,masc,Sgtm,Surn sing,nomn
True


In [28]:
p = morph.parse('Петрович')[0].tag
print('Тэги - ', p)
print('Patr' in p) #тэг отчества

Тэги -  NOUN,anim,masc,Patr sing,nomn
True


In [29]:
p = morph.parse('Москва')[0].tag
print('Тэги - ', p)
print('Geox' in p) #тэг локация

Тэги -  NOUN,inan,femn,Sgtm,Geox sing,nomn
True


In [32]:
p = morph.parse('Яндекс')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  NOUN,inan,masc,Orgn sing,nomn
True


In [68]:
p = morph.parse('Вшэ')[0].tag
print('Тэги - ', p)
print('Orgn' in p) #тэг организация

Тэги -  UNKN
False


Работает не очень хорошо, но все равно лучше, чем ничего. Рядом стоящие слова одного тэга можно склеить в один. Или сначала собрать нграмы и если какое-то одно слово в нграмме принадлежит к какому-то типу сущности, то распространить его на весь нграм.

Есть пара библотек, специально предназначенных для этого. Например, natasha - https://github.com/natasha/natasha

Она основана на парсере yargy https://github.com/natasha/yargy и представляет собой набор готовых правил для извлечения некоторых сущностей.

In [2]:
from natasha import (NamesExtractor,
                     SimpleNamesExtractor,
                     PersonExtractor,
                     LocationExtractor,
                     AddressExtractor,
                     OrganisationExtractor,
                     DatesExtractor,
                     MoneyExtractor,
                     MoneyRateExtractor,
                     MoneyRangeExtractor)

from natasha.markup import (show_markup_notebook as show_markup,
                            format_json)

In [103]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = NamesExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [102]:
text = 'Влад Веселов. Петрович. Алиса. Студия Артемия Лебедева'

extractor_per = PersonExtractor()
matches = extractor_per(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [101]:
text = 'Более того в Москве, в районе Строгино. На реке Оке. В германии'

extractor_loc = LocationExtractor()
matches = extractor_loc(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [97]:
text = 'ФСБ. Московский государственный университет. Высшая школа экономика. ВШЭ. Mail.ru'

extractor_org = OrganisationExtractor()
matches = extractor_org(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [96]:
text = 'С 2015 по 2017 год. 16 апреля 1993 года. В субботу. 23.04.18'

extractor_date = DatesExtractor()
matches = extractor_date(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

In [94]:
text = "Он заплатил ему 300 рублей."

extractor_money = MoneyExtractor()
matches = extractor_money(text)
spans = [_.span for _ in matches]
facts = [_.fact.as_json for _ in matches]
show_markup(text, spans)
# print(format_json(facts))

В yargy можно писать свои грамматики, подробнее про синтаксис можно почитать в: http://yargy.readthedocs.io/ru/latest/

Ещё есть томита-парсер, но с ним очень тяжело работать (никакого развития, скудная документация, закрытый код, никакого сообщества) https://tech.yandex.ru/tomita/

Для русского state-of-the-art - библеотека от Ipavlov из Физтеха.
https://github.com/deepmipt/ner

Она основана на BiLSTM-CRF (нейронки) и скорее всего просто так не поставится и использовать её будет трудновато (без GPU).

In [1]:
import ner
extractor = ner.Extractor()

100%|██████████| 44.3M/44.3M [00:03<00:00, 11.8MB/s]


Extracting /usr/local/lib/python3.5/dist-packages/ner/extractor/../model/ner_model_total_rus.tar.gz archive into /usr/local/lib/python3.5/dist-packages/ner/extractor/../model
Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead


/usr/local/lib/python3.5/dist-packages/tensorflow/python/ops/gradients_impl.py:98: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Restoring parameters from /usr/local/lib/python3.5/dist-packages/ner/extractor/../model/ner_model


In [7]:
list(extractor('ФСБ. Московский государственный университет. \
               Высшая школа экономика. ВШЭ. Mail.ru'))

[Match(tokens=[Token(span=(0, 3), text='ФСБ')], span=Span(start=0, end=3), type='ORG'),
 Match(tokens=[Token(span=(5, 15), text='Московский'), Token(span=(16, 31), text='государственный'), Token(span=(32, 43), text='университет')], span=Span(start=5, end=43), type='ORG')]